In [1034]:
import cv2 as cv
import mediapipe as mp
import time
import matplotlib.pyplot as plt
import math

In [1035]:
def find_distance(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [1036]:
# def find_angle(x1, y1, x2, y2):
#     theta = math.acos((y2-y1)*(-y1) / (math.sqrt((x2-x1)**2 + (y2-y1)**2) * y1))
#     degree = int(180/math.pi) * theta

#     return degree

def find_angle(x1, y1, x2, y2):
    theta = math.atan2(y2 - y1, x2 - x1)
    degree = int(math.degrees(theta))

    if degree < 0:
        degree += 180

    return degree

In [1037]:
def find_midpoint(x1, y1, x2, y2):
    mx = int((x1 + x2) / 2)
    my = int((y1 + y2) / 2)
    return mx, my

In [1038]:
def send_warning(x):
    pass

In [1039]:
import numpy as np

def draw_angle_indicator(image, center, radius, start_angle, end_angle, color, thickness):
    # Convert angles to radians
    start_angle_rad = math.radians(start_angle)
    end_angle_rad = math.radians(end_angle)

    # Calculate start and end points of the arc
    start_point = (
        int(center[0] + radius * np.cos(start_angle_rad)),
        int(center[1] + radius * np.sin(start_angle_rad))
    )
    end_point = (
        int(center[0] + radius * np.cos(end_angle_rad)),
        int(center[1] + radius * np.sin(end_angle_rad))
    )

    # Draw the arc on the image
    cv.ellipse(image, center, (radius, radius), 0, start_angle - 180, end_angle - 180, color, thickness)

    # Draw lines connecting the center to the start and end points
    cv.line(image, center, start_point, color, thickness)
    cv.line(image, center, end_point, color, thickness)

In [1040]:
def calc_neck_inclination(lm, lmPose, w, h, frame, font, colors, fps, good_frames, bad_frames):
    red, green, light_green, yellow, pink = colors

    # Acquire the landmark coordinates.
    # Once aligned properly, left or right should not be a concern.
    # Left shoulder.
    l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
    l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
    # Right shoulder
    r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
    r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
    # Left ear.
    l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
    l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
    # Left hip.
    l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
    l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)
    # Left outer eye
    l_eye_x = int(lm.landmark[lmPose.LEFT_EYE_OUTER].x * w)
    l_eye_y = int(lm.landmark[lmPose.LEFT_EYE_OUTER].y * h)
    
    # Get tragus
    eye_to_ear = find_distance(l_eye_x, l_eye_y, l_ear_x, l_ear_y)
    trg_x, trg_y = int(l_ear_x + (eye_to_ear / 2)), int(l_ear_y - (eye_to_ear / 4))

    cv.circle(frame, (trg_x, trg_y), 7, pink, -1)

    # Calculate distance between left shoulder and right shoulder points.
    offset = find_distance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
    
    # Get midpoint of left and right shoulder points.
    mx, my = find_midpoint(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

    # Calculate distance between tragus and midpoint.
    trg_to_mid = find_distance(trg_x, trg_y, mx, my)
    neck_ratio = 0.25
    c7_x, c7_y = int(mx + (trg_to_mid * neck_ratio)), int(my - (trg_to_mid * neck_ratio))
    
    cv.circle(frame, (c7_x, c7_y), 7, pink, -1)
    cv.circle(frame, (mx, my), 7, yellow, -1)
    
    # Draw imaginary horizontal line from midpoint.
    cv.line(frame, (c7_x - 300, c7_y), (c7_x + 300, c7_y), yellow, 2)
    cv.circle(frame, (c7_x - 300, c7_y), 7, yellow, -1)
    cv.circle(frame, (c7_x + 300, c7_y), 7, yellow, -1)
    
    # Connect shoulder to c7 point
    cv.line(frame, (l_shldr_x, l_shldr_y), (c7_x, c7_y), yellow, 2)
    cv.line(frame, (r_shldr_x, r_shldr_y), (c7_x, c7_y), yellow, 2)
    
    # Draw imaginary vertical line from midpoint.
    cv.line(frame, (c7_x, c7_y - 50), (c7_x, c7_y + 50), yellow, 2)
    
    cv.putText(frame, str(mx) + ' ' + str(my), (w - 150, 60), font, 0.9, green, 2)
    # Assist to align the camera to point at the side view of the person.
    # Offset threshold 30 is based on results obtained from analysis over 100 samples.
    if offset < 100:
        cv.putText(frame, str(int(offset)) + ' Aligned', (w - 150, 30), font, 0.9, green, 2)
    else:
        cv.putText(frame, str(int(offset)) + ' Not Aligned', (w - 150, 30), font, 0.9, red, 2)
    
    # Calculate angles.
    neck_inclination = find_angle(c7_x, c7_y, trg_x, trg_y)

    # Draw landmarks.
    cv.circle(frame, (l_shldr_x, l_shldr_y), 7, green, -1)

    # cv.circle(frame, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
    cv.circle(frame, (r_shldr_x, r_shldr_y), 7, red, -1)
    cv.circle(frame, (l_hip_x, l_hip_y), 7, yellow, -1)

    draw_angle_indicator(frame, (c7_x, c7_y), 50, 0, neck_inclination, yellow, 2)

    # Put text, Posture and angle inclination.
    # Text string for display.
    angle_text_string = 'Neck angle : ' + str(int(neck_inclination))

    # Determine whether good posture or bad posture.
    # The threshold angles have been set based on intuition.
    if neck_inclination >= 48 and neck_inclination <= 90:
        cv.putText(frame, angle_text_string, (10, 30), font, 0.9, light_green, 2)
        cv.putText(frame, str(int(neck_inclination)), (c7_x + 10, c7_y - 10), font, 0.9, light_green, 2)

        # Join landmarks.
        cv.line(frame, (c7_x, c7_y), (trg_x, trg_y), green, 4)
        # cv.line(frame, (mx, my), (mx, my - 100), green, 4)
        # cv.line(frame, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), green, 4)
        # cv.line(frame, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), green, 4)

    else:
        cv.putText(frame, angle_text_string, (10, 30), font, 0.9, red, 2)
        cv.putText(frame, str(int(neck_inclination)), (c7_x + 10, c7_y - 10), font, 0.9, red, 2)

        # Join landmarks.
        cv.line(frame, (c7_x, c7_y), (trg_x, trg_y), red, 4)
        # cv.line(frame, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), red, 4)
        # cv.line(frame, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), red, 4)
        # cv.line(frame, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), red, 4)

In [1041]:
def add_meta_info(frame, text, font, font_scale, color, thickness, vertical_position):
    # Get the frame size
    frame_height, frame_width, _ = frame.shape

    # Get the size of the text
    text_size = cv.getTextSize(text, font, font_scale, thickness)[0]

    # Calculate the position to place the text on the bottom right corner
    text_position = (frame_width - text_size[0] - 10, frame_height - vertical_position)

    # Adding text to the frame
    cv.putText(frame, text, text_position, font, font_scale, color, thickness, cv.LINE_AA)

In [1042]:
# Initilize frame counters.
good_frames = 0
bad_frames = 0

# Font type.
font = cv.FONT_HERSHEY_SIMPLEX

# Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

colors = (red, green, light_green, yellow, pink)

In [1043]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_draw = mp.solutions.drawing_utils
mp_draw_style = mp.solutions.drawing_styles

cap = cv.VideoCapture(1)

# Meta
fps = int(cap.get(cv.CAP_PROP_FPS))
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)

In [1044]:
print('Processing..')

if not cap.isOpened():
    print('Cannot open camera.')
    exit()

while True:
    # Capture frames.
    success, frame = cap.read()
    if not success:
        print("Error: Failed to capture frame.")
        break
    
    add_meta_info(frame, "FPS: " + str(fps), font, 1, blue, 2, 20)
    add_meta_info(frame, "Frame size: " + str(frame.shape[1]) + "x" + str(frame.shape[0]), font, 1, blue, 2, 60)
    
    # Get height and width.
    h, w = frame.shape[:2]

    # Convert the BGR frame to RGB.
    imgRBG = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    
    # Improve performance
    frame.flags.writeable = True

    # Process the image.
    keypoints = pose.process(imgRBG)

    # Use lm and lmPose as representative of the following methods.
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    if lm:
        calc_neck_inclination(lm, lmPose, w, h, frame, font, colors, fps, good_frames, bad_frames)
        
    # mp_draw.draw_landmarks(frame, lm, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_draw_style.get_default_pose_landmarks_style())
        
    cv.imshow('cam', frame)

    k = cv.waitKey(1)
    if k & 0xFF == 27: break #27 is ESC key.
    
    
print('Finished.')
cap.release()
cv.destroyAllWindows()

Processing..
Finished.
